In [1]:
%pwd

'/home/group.kurse/cviwo013/ComputerVisionProject'

In [2]:
import numpy as np
import os
from pathlib import Path
from PIL import Image
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms as T
import torch
import random
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from CellDataset import CellDataset
from MoCoResNetBackbone import MoCoResNetBackbone
from MoCoV2Loss import MoCoV2Loss

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = MoCoResNetBackbone()
model.to(device)

dataset = CellDataset()

moco_loss = MoCoV2Loss(device=device)

epochs = 20
batch_size = 32
learning_rate = 0.001
momentum = 0.9

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=learning_rate,
    momentum=momentum
)

dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,        # Adjust to CPU core count
    pin_memory=True,       # Enables fast transfer to GPU
)

losses = [[] for _ in range(epochs)]

for epoch in range(epochs):
    model.train()
    for [keys, queries] in tqdm(dataloader, desc=f"Epoch {epoch}", total=len(dataloader), ncols=100):

        keys = keys.to(device, non_blocking=True)
        queries = queries.to(device, non_blocking=True)

        query_encodings, key_encodings = model(queries, keys)

        loss = moco_loss(query_encodings, key_encodings)
        losses[epoch].append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print(f"Epoch {epoch} loss: {sum(losses[epoch]) / len(losses[epoch])} ")


Loaded keys: _IncompatibleKeys(missing_keys=['0.weight', '1.weight', '1.bias', '1.running_mean', '1.running_var', '4.0.conv1.weight', '4.0.bn1.weight', '4.0.bn1.bias', '4.0.bn1.running_mean', '4.0.bn1.running_var', '4.0.conv2.weight', '4.0.bn2.weight', '4.0.bn2.bias', '4.0.bn2.running_mean', '4.0.bn2.running_var', '4.0.conv3.weight', '4.0.bn3.weight', '4.0.bn3.bias', '4.0.bn3.running_mean', '4.0.bn3.running_var', '4.0.downsample.0.weight', '4.0.downsample.1.weight', '4.0.downsample.1.bias', '4.0.downsample.1.running_mean', '4.0.downsample.1.running_var', '4.1.conv1.weight', '4.1.bn1.weight', '4.1.bn1.bias', '4.1.bn1.running_mean', '4.1.bn1.running_var', '4.1.conv2.weight', '4.1.bn2.weight', '4.1.bn2.bias', '4.1.bn2.running_mean', '4.1.bn2.running_var', '4.1.conv3.weight', '4.1.bn3.weight', '4.1.bn3.bias', '4.1.bn3.running_mean', '4.1.bn3.running_var', '4.2.conv1.weight', '4.2.bn1.weight', '4.2.bn1.bias', '4.2.bn1.running_mean', '4.2.bn1.running_var', '4.2.conv2.weight', '4.2.bn2.weight

Epoch 0:  33%|████████████████▍                                 | 1036/3158 [01:41<03:25, 10.31it/s]